# Main

In [3]:
import pandas as pd
import numpy as np
import re
import random
from sklearn.model_selection import train_test_split
import json

from gensim.models import Word2Vec
import string
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
from utils import clean_text, text2words

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch.optim as torch_optim
import torch.nn.functional as F
import pandas as pd
from sklearn import metrics

from model import MODEL

## Load the data

In [4]:
df = pd.read_csv('alldata_1_for_kaggle_utf8.csv', on_bad_lines='skip')

In [5]:
df

,Unnamed: 0,0,a
0,0,Thyroid_Cancer,Thyroid surgery in children in a single insti...
1,1,Thyroid_Cancer,""" The adopted strategy was the same as that us..."
2,2,Thyroid_Cancer,coronary arterybypass grafting thrombosis Ô¨Åb...
3,3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...
4,4,Thyroid_Cancer,This study aimed to investigate serum matrix ...
...,...,...,...
7565,7565,Colon_Cancer,we report the case of a 24yearold man who pres...
7566,7566,Colon_Cancer,among synchronous colorectal cancers scrcs rep...
7567,7567,Colon_Cancer,NaN
7568,7568,Colon_Cancer,""""


In [6]:
df = df.rename(columns={'Unnamed: 0':'ID', '0':'cancer_type', 'a':'note_text'})

In [7]:
df = df.loc[~df.note_text.isnull()]

In [8]:
df

,ID,cancer_type,note_text
0,0,Thyroid_Cancer,Thyroid surgery in children in a single insti...
1,1,Thyroid_Cancer,""" The adopted strategy was the same as that us..."
2,2,Thyroid_Cancer,coronary arterybypass grafting thrombosis Ô¨Åb...
3,3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...
4,4,Thyroid_Cancer,This study aimed to investigate serum matrix ...
...,...,...,...
7564,7564,Colon_Cancer,""""
7565,7565,Colon_Cancer,we report the case of a 24yearold man who pres...
7566,7566,Colon_Cancer,among synchronous colorectal cancers scrcs rep...
7568,7568,Colon_Cancer,""""


### Map the targets

In [9]:
df = pd.concat([df, pd.get_dummies(df['cancer_type'])], axis=1)

In [10]:
df

,ID,cancer_type,note_text,Colon_Cancer,Lung_Cancer,Thyroid_Cancer
0,0,Thyroid_Cancer,Thyroid surgery in children in a single insti...,0,0,1
1,1,Thyroid_Cancer,""" The adopted strategy was the same as that us...",0,0,1
2,2,Thyroid_Cancer,coronary arterybypass grafting thrombosis Ô¨Åb...,0,0,1
3,3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...,0,0,1
4,4,Thyroid_Cancer,This study aimed to investigate serum matrix ...,0,0,1
...,...,...,...,...,...,...
7564,7564,Colon_Cancer,"""",1,0,0
7565,7565,Colon_Cancer,we report the case of a 24yearold man who pres...,1,0,0
7566,7566,Colon_Cancer,among synchronous colorectal cancers scrcs rep...,1,0,0
7568,7568,Colon_Cancer,"""",1,0,0


In [35]:
df['Colon_Cancer'] = df.Colon_Cancer*2

<ipython-input-35-0925a89ffc8b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Colon_Cancer'] = df.Colon_Cancer*2


In [12]:
df['Lung_Cancer'] = df.Lung_Cancer*3

In [13]:
df['target'] = df['Colon_Cancer']+df['Lung_Cancer']+df['Thyroid_Cancer']-1

Targets - 0: Thyroid, 1: Colon, 2:Lung

In [36]:
df

,ID,cancer_type,note_text,Colon_Cancer,Lung_Cancer,Thyroid_Cancer,target,cleaned,text
0,0,Thyroid_Cancer,Thyroid surgery in children in a single insti...,0,0,1,0,thyroid surgery in children in a single instit...,"[thyroid, surgery, in, children, in, a, single..."
1,1,Thyroid_Cancer,""" The adopted strategy was the same as that us...",0,0,1,0,the adopted strategy was the same as that used...,"[the, adopted, strategy, was, the, same, as, t..."
2,2,Thyroid_Cancer,coronary arterybypass grafting thrombosis Ô¨Åb...,0,0,1,0,coronary arterybypass grafting thrombosis brin...,"[coronary, arterybypass, grafting, thrombosis,..."
3,3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...,0,0,1,0,solitary plasmacytoma sp of the skull is an un...,"[solitary, plasmacytoma, sp, of, the, skull, i..."
4,4,Thyroid_Cancer,This study aimed to investigate serum matrix ...,0,0,1,0,this study aimed to investigate serum matrix m...,"[this, study, aimed, to, investigate, serum, m..."
...,...,...,...,...,...,...,...,...,...
7564,7564,Colon_Cancer,"""",4,0,0,1,,[]
7565,7565,Colon_Cancer,we report the case of a 24yearold man who pres...,4,0,0,1,we report the case of a 0 yearold man who pres...,"[we, report, the, case, of, a, 0, yearold, man..."
7566,7566,Colon_Cancer,among synchronous colorectal cancers scrcs rep...,4,0,0,1,among synchronous colorectal cancers scrcs rep...,"[among, synchronous, colorectal, cancers, scrc..."
7568,7568,Colon_Cancer,"""",4,0,0,1,,[]


## Clean the text

In [15]:
df['cleaned'] = df.note_text.apply(lambda x: clean_text(x))
df['text'] = df.cleaned.str.split(r'\s+')

### Output the targets as JSON

In [16]:
target_dict = {}
for idx, row in df[['ID', 'target']].drop_duplicates().iterrows():
    target_dict[str(row['ID'])] = str(row['target'])
json.dump(target_dict, open('./target_dict.json', 'w'))


# Try 1: Classify Thyroid vs. Colon

In [17]:
df = df.loc[df.target.isin([0,1])]

In [18]:
df

,ID,cancer_type,note_text,Colon_Cancer,Lung_Cancer,Thyroid_Cancer,target,cleaned,text
0,0,Thyroid_Cancer,Thyroid surgery in children in a single insti...,0,0,1,0,thyroid surgery in children in a single instit...,"[thyroid, surgery, in, children, in, a, single..."
1,1,Thyroid_Cancer,""" The adopted strategy was the same as that us...",0,0,1,0,the adopted strategy was the same as that used...,"[the, adopted, strategy, was, the, same, as, t..."
2,2,Thyroid_Cancer,coronary arterybypass grafting thrombosis Ô¨Åb...,0,0,1,0,coronary arterybypass grafting thrombosis brin...,"[coronary, arterybypass, grafting, thrombosis,..."
3,3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...,0,0,1,0,solitary plasmacytoma sp of the skull is an un...,"[solitary, plasmacytoma, sp, of, the, skull, i..."
4,4,Thyroid_Cancer,This study aimed to investigate serum matrix ...,0,0,1,0,this study aimed to investigate serum matrix m...,"[this, study, aimed, to, investigate, serum, m..."
...,...,...,...,...,...,...,...,...,...
7564,7564,Colon_Cancer,"""",2,0,0,1,,[]
7565,7565,Colon_Cancer,we report the case of a 24yearold man who pres...,2,0,0,1,we report the case of a 0 yearold man who pres...,"[we, report, the, case, of, a, 0, yearold, man..."
7566,7566,Colon_Cancer,among synchronous colorectal cancers scrcs rep...,2,0,0,1,among synchronous colorectal cancers scrcs rep...,"[among, synchronous, colorectal, cancers, scrc..."
7568,7568,Colon_Cancer,"""",2,0,0,1,,[]


## Make the train/valid/test splits

In [19]:
X_train, X_test, y_train, y_test = train_test_split(df[['ID', 'note_text', 'cleaned', 'text']], df[['ID', 'target']], test_size=0.2, random_state=1)

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.25, random_state=1)

In [21]:
X_train

,ID,note_text,cleaned,text
2744,2744,"""development of cancer is a problem that has a...",development of cancer is a problem that has ac...,"[development, of, cancer, is, a, problem, that..."
519,519,"""it is well understood that the level of molec...",it is well understood that the level of molecu...,"[it, is, well, understood, that, the, level, o..."
4286,4286,bousmalis leveraged cgan with the contents...,bousmalis leveraged cgan with the contentsimil...,"[bousmalis, leveraged, cgan, with, the, conten..."
5846,5846,""" mirnas regulate a multitude of cellular pro...",mirnas regulate a multitude of cellular proces...,"[mirnas, regulate, a, multitude, of, cellular,..."
270,270,Integrinmediated adhesive properties ofneutro...,integrinmediated adhesive properties ofneutrop...,"[integrinmediated, adhesive, properties, ofneu..."
...,...,...,...,...
7435,7435,sarscov2 has resulted in numerous cases of cor...,sarscov 0 has resulted in numerous cases of co...,"[sarscov, 0, has, resulted, in, numerous, case..."
1093,1093,molecular heterogeneity of renal cell carcinom...,molecular heterogeneity of renal cell carcinom...,"[molecular, heterogeneity, of, renal, cell, ca..."
4788,4788,"""protein protein interaction screen carried ou...",protein protein interaction screen carried out...,"[protein, protein, interaction, screen, carrie..."
239,239,""" formulated a traditional Chinese medicin...",formulated a traditional chinese medicine tcm ...,"[formulated, a, traditional, chinese, medicine..."


## Run the Word2Vec Model

In [22]:
vector_size = 50
num_words = 1000

In [23]:
def word2vec_model(texts, vector_size=50):
    
    print('Running the Word2Vec Model...\n\n')
    
    model = Word2Vec(sentences=texts, size=vector_size, min_count=1)
    
    return model

In [40]:
model = word2vec_model(X_train.text.tolist(), vector_size)

Running the Word2Vec Model...




In [41]:
y_train

,ID,target
2744,2744,0
519,519,1
4286,4286,1
5846,5846,1
270,270,0
...,...,...
7435,7435,1
1093,1093,0
4788,4788,0
239,239,0


In [42]:
y_train.target.unique()

array([0, 1], dtype=uint8)

In [43]:
my_dict = dict({})
for idx, key in enumerate(model.wv.vocab):
    my_dict[key] = model.wv[key]

## Dataset Class

In [44]:
class DataBowl(Dataset):
    def __init__(self, X, y, ids, vector_size, my_dict, num_words, phase='train'):
        assert (phase == 'train' or phase == 'valid' or phase == 'test')
        self.phase = phase
        self.X = X
        self.y = y
        self.ids = ids
        self.vector_size = vector_size
        self.my_dict = my_dict
        self.num_words = num_words

    def get_mm_item(self, idx):

        pid = self.ids.iloc[idx]
        label = np.array([int(l)
                          for l in self.y.loc[self.y.ID==pid].target], dtype=np.float32)        
#         content = model[X.loc[X.ID==pid].iloc[0]['text'][-1000:]]

        content = [my_dict.get(k, np.zeros(50,dtype=float)) for k in self.X.loc[self.X.ID==pid]['text']\
                   .iloc[0][-self.num_words:]]
        
        if len(content) < self.num_words:
            y = self.num_words-len(content)
            add_ = []
            for j in range(y):
                add_.append([0]*self.vector_size)

            add_.extend(content)
            content = add_
        else:
            content = content[-self.num_words:]
        
        content = np.array(content, dtype=np.float32)

#         return torch.from_numpy(input_list), torch.from_numpy(time_list), torch.from_numpy(demo), torch.from_numpy(content), torch.from_numpy(label), input_file

        return torch.from_numpy(content), torch.from_numpy(label), pid

    def __getitem__(self, idx):
        return self.get_mm_item(idx)

    def __len__(self):
        return len(self.ids)

## Initiate the data sets

In [45]:
train_ds = DataBowl(X_train, y_train, X_train.ID, vector_size, my_dict, num_words, phase='train')
valid_ds = DataBowl(X_valid, y_valid, X_valid.ID, vector_size, my_dict, num_words, phase='valid')
test_ds = DataBowl(X_test, y_test, X_test.ID, vector_size, my_dict, num_words, phase='test')

In [46]:
y_train

,ID,target
2744,2744,0
519,519,1
4286,4286,1
5846,5846,1
270,270,0
...,...,...
7435,7435,1
1093,1093,0
4788,4788,0
239,239,0


## Support Functions

In [47]:
def save_model(p_dict):
    model = p_dict['model']
    state_dict = model.state_dict()
    for key in state_dict.keys():
        state_dict[key] = state_dict[key].cpu()
    all_dict = {
            'best_metric': p_dict['best_metric'],
            'state_dict': state_dict 
            }
    torch.save(all_dict, 'best_model.ckpt')

In [48]:
def load_model(p_dict):
    all_dict = torch.load('best_model.ckpt')
    p_dict['best_metric'] = all_dict['best_metric']
    p_dict['model'].load_state_dict(all_dict['state_dict'])

## Train the model

In [53]:
model = MODEL({'K':20})

parameters_all = []
for p in model.parameters():
    parameters_all.append(p)

optimizer = torch.optim.Adam(parameters_all)

loss = F.binary_cross_entropy

def train_eval(data_loader, optimizer, loss, phase, saliency=False, best_metric=0, optimal_threshold=0):
    
#     for e in range(epochs):
        
    if saliency:
        model.eval()
        
        df = pd.DataFrame(columns=range(20))
        
    else: 
        if phase == 'train':
            model.train()
        else:
            model.eval()
    
    loss_list, pred_list, label_list, = [], [], []

    for b, data_list in enumerate(data_loader):
        content, label, patients = data_list
        
        content = content.view(content.shape[0],1,num_words,vector_size)
        output_global, output_local, output_saliency = model(content)

        
        if saliency:
            
            temp_df = pd.DataFrame(index=patients.numpy(), columns=range(20))
            for iii in range(len(patients)):
                temp_saliency = [i.item() for i in output_saliency[iii,:,0]]
                temp_df.loc[patients[iii].item()] = temp_saliency

            df = df.append(temp_df)
            
            df.to_csv('saliency_results_'+phase+'.csv')
                
        else:
            
            if phase=='train':
                    
                loss_output = loss(output_local, label)

                optimizer.zero_grad()
                loss_output.backward()
                optimizer.step()

            elif phase=='valid':

                loss_output = loss(output_local, label)

            elif phase=='test':

                loss_output = loss(output_local, label)

            output = output_local

            pred_list.append(output.data.cpu().numpy())
            loss_list.append(loss_output.data.cpu().numpy())
            label_list.append(label.data.cpu().numpy())

    if saliency==False:
    
        if phase=='valid':

            label_list = [jj[0] for jj in label_list]
            pred_list = [jj[0] for jj in label_list]

            # Calculate the AUC

            fpr, tpr, thresholds = metrics.roc_curve(label_list, pred_list)
            auc = metrics.auc(fpr, tpr)

            # Calculate the optimal threshold by f1 score
            preds = [1 if prob >= thresholds[0] else 0 for prob in pred_list]
            f1 = metrics.f1_score(label_list, preds)

            for t in thresholds[1:]:
                preds = [1 if prob >= t else 0 for prob in pred_list]
                temp_f1 = metrics.f1_score(label_list, preds)
                if temp_f1 > f1:
                    optimal_threshold = t
                    f1 = temp_f1
            preds = [1 if prob >= optimal_threshold else 0 for prob in pred_list]
            tn, fp, fn, tp = metrics.confusion_matrix(label_list, preds).ravel()
            precision = 1.0 * (tp / (tp + fp))
            sen = 1.0 * (tp / (tp + fn))
            spec = 1.0 * (tn / (tn + fp))
            f1 = metrics.f1_score(label_list, preds)

            print('Validation AUC: {:}\n'.format(auc))
            print('Validation Precision: {:}\n'.format(precision))
            print('Validation Sensitivity: {:}\n'.format(sen))
            print('Validation Specificity: {:}\n'.format(spec))
            print('Validation F1: {:}\n'.format(f1))

            if f1>best_metric:
                best_metric = f1
                save_model({'model': model, 'best_metric': best_metric})
                print('Saving the model')
            return best_metric, optimal_threshold


        elif phase=='test':
            label_list = [jj[0] for jj in label_list]
            pred_list = [jj[0] for jj in label_list]

            # Calculate the AUC
            fpr, tpr, thresholds = metrics.roc_curve(label_list, pred_list)
            auc = metrics.auc(fpr, tpr)

            # Calculate the optimal threshold by f1 score

            preds = [1 if prob >= optimal_threshold else 0 for prob in pred_list]
            tn, fp, fn, tp = metrics.confusion_matrix(label_list, preds).ravel()
            precision = 1.0 * (tp / (tp + fp))
            sen = 1.0 * (tp / (tp + fn))
            spec = 1.0 * (tn / (tn + fp))
            f1 = metrics.f1_score(label_list, preds)

            print('Test AUC: {:}\n'.format(auc))
            print('Test Precision: {:}\n'.format(precision))
            print('Test Sensitivity: {:}\n'.format(sen))
            print('Test Specificity: {:}\n'.format(spec))
            print('Test F1: {:}\n'.format(f1))

            return auc, precision, sen, spec, f1
        

### Train the model

In [54]:
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_ds, batch_size=1, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=1, shuffle=False)

In [55]:
epochs = 20
best_metric = 0

for i in range(epochs):
    print('\n\n#################\n\nEpoch: '+str(i)+'\n\n#################\n')
    train_eval(train_loader, optimizer, loss, 'train')
    best_metric,optimal_threshold = train_eval(valid_loader, optimizer, loss, 'valid', best_metric=best_metric)
    train_eval(test_loader, optimizer, loss, 'test', best_metric=best_metric, optimal_threshold=optimal_threshold)



#################

Epoch: 0

#################

Validation AUC: 1.0

Validation Precision: 1.0

Validation Sensitivity: 1.0

Validation Specificity: 1.0

Validation F1: 1.0

Saving the model
Test AUC: 1.0

Test Precision: 1.0

Test Sensitivity: 1.0

Test Specificity: 1.0

Test F1: 1.0



#################

Epoch: 1

#################

Validation AUC: 1.0

Validation Precision: 1.0

Validation Sensitivity: 1.0

Validation Specificity: 1.0

Validation F1: 1.0

Test AUC: 1.0

Test Precision: 1.0

Test Sensitivity: 1.0

Test Specificity: 1.0

Test F1: 1.0



#################

Epoch: 2

#################

Validation AUC: 1.0

Validation Precision: 1.0

Validation Sensitivity: 1.0

Validation Specificity: 1.0

Validation F1: 1.0



KeyboardInterrupt: 

## Extract Local Texts

### Just take a random sample for the saliency

In [56]:
train_ids = random.sample(X_train.ID.tolist(), 100)
test_ids = random.sample(X_test.ID.tolist(),100)

train_ds_sal = DataBowl(X_train.loc[X_train.ID.isin(train_ids)], \
                        y_train.loc[y_train.ID.isin(train_ids)], pd.Series(train_ids), vector_size, my_dict, num_words, phase='train')
test_ds_sal = DataBowl(X_test.loc[X_test.ID.isin(test_ids)], \
                       y_test.loc[y_test.ID.isin(test_ids)], pd.Series(test_ids), vector_size, my_dict, num_words, phase='test')
                                    

In [57]:
train_loader_saliency = DataLoader(train_ds_sal, batch_size=1)
test_loader_saliency  = DataLoader(test_ds_sal, batch_size=1)

In [58]:
train_eval(train_loader_saliency, optimizer, loss, 'train', saliency=True)

In [59]:
train_eval(test_loader_saliency, optimizer, loss, 'test', saliency=True)

## Analyze the saliency texts

In [60]:
sal_train = pd.read_csv('saliency_results_train.csv').rename(columns={'Unnamed: 0':'ID'}).set_index('ID')

In [61]:
sal_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
ID,,,,,,,,,,,,,,,,,,,,
1812,191,61,800,85,27,762,220,306,176,682,987,42,661,940,163,840,331,709,423,614
2926,574,54,694,5,597,844,458,198,651,823,24,76,678,805,958,491,746,662,710,223
1272,979,117,370,221,340,388,938,814,20,266,305,617,736,892,70,468,963,786,662,173
530,427,592,413,772,881,507,650,493,133,103,561,713,8,88,724,62,49,536,760,462
238,539,223,958,611,377,938,529,686,508,183,17,775,315,416,923,851,908,736,559,667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4022,398,957,595,763,639,794,74,745,556,177,582,857,986,151,778,459,15,701,542,490
7340,240,274,304,338,255,936,829,593,138,43,166,122,81,199,293,909,111,725,59,356
1008,197,728,664,901,918,796,987,870,534,92,134,637,179,360,235,158,335,940,619,551


## Print local texts

In [67]:
id1 = sal_train.index[4]

temp_note = df.loc[df.ID==id1]['text'].iloc[0]

if len(temp_note)<num_words:
    print(len(temp_note))
    ddiff = num_words-len(temp_note)
    buffer = ['' for i in range(ddiff)]
    buffer.extend(temp_note)
    
    temp_note = buffer
    
    
print(df.loc[df.ID==id1].cancer_type.iloc[0])
    
for ii in sal_train.loc[id1]:
    print(' '.join(temp_note[ii-5:ii+15]))
    

Thyroid_Cancer
the same country such as the living conditions the healthrelated behaviour khalatbarisoltani et al corresponding author email address ccopatunictit c
it determines important limitations for direct comparison of results and more studies are needed to strengthen scientific evidences and support
a worsening factor of the covid 0 severity conticini frontera isaifan martelletti and martelletti this association is getting stronger thanks
among the environmental parameters some climate condition such as temperature humidity sunlight and wind revealed a reduction of the covid
because of their reduced ventilation morawska lack ultraviolet light which rapidly inactivates the virus and because it can become less
data level of industrialization as well as regional topography could operate both as a carrier of the infection and as
0 in the diverse parts of the world or of the same country such as the living conditions the healthrelated
as the largest environmental cause of disease an

In [33]:
sal_test = pd.read_csv('saliency_results_test.csv').rename(columns={'Unnamed: 0':'ID'}).set_index('ID')

In [34]:
id1 = sal_test.index[0]

temp_note = df.loc[df.ID==id1]['text'].iloc[0]

if len(temp_note)<num_words:
    print(len(temp_note))
    ddiff = num_words-len(temp_note)
    buffer = ['' for i in range(ddiff)]
    buffer.extend(temp_note)
    
    temp_note = buffer
    
    
print(df.loc[df.ID==id1].target.iloc[0])
    
for ii in sal_test.loc[id1]:
    print(' '.join(temp_note[ii-5:ii+15]))
    

111
0
 i watched this show and i simply didn t find it funny at all it might have been the
as a station all the characters on this show are pretty bad actors but even if they were good the
they are playing they might just keep this one simply because it s average compared to them
good the jokes and script are pretty horrible and would still bring the show down i would say that i
the show down i would say that i believe this show will be cancelled but seeing as how abc is
at all it might have been the first episode lately i realize abc is playing a lot of stupid shows
be cancelled but seeing as how abc is doing pretty horrible for quality of shows they are playing they might
is playing a lot of stupid shows nowadays and is going down as a station all the characters on this
               i watched this show and

                   
                   
                   
                   
                   
                   
                   
                   
                 